In [37]:
import pandas as pd
from citipy import citipy
import csv


In [12]:
airbnb = pd.read_csv("../../train.csv")


In [13]:
airbnb.head(20)
lat = airbnb['latitude']
lon = airbnb['longitude']
cities = []
state=[]

lat_lng = zip(lat, lon)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lng:
    
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    cities.append(city)

# Print the city count to confirm sufficient count
print(city)

long beach


In [17]:
airbnb['cities'] = cities
airbnb['cities'].value_counts()

new york          20344
san francisco      5505
west hollywood     5144
edgewater          5076
washington         4100
                  ...  
tiburon               1
summit                1
walnut park           1
elizabeth             1
south gate            1
Name: cities, Length: 164, dtype: int64

In [27]:
airbnb['city'].value_counts()

NYC        32349
LA         22453
SF          6434
DC          5688
Chicago     3719
Boston      3468
Name: city, dtype: int64

In [19]:
airbnb.columns

Index(['id', 'log_price', 'property_type', 'room_type', 'amenities',
       'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy',
       'cleaning_fee', 'city', 'description', 'first_review',
       'host_has_profile_pic', 'host_identity_verified', 'host_response_rate',
       'host_since', 'instant_bookable', 'last_review', 'latitude',
       'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'review_scores_rating', 'thumbnail_url', 'zipcode', 'bedrooms', 'beds',
       'cities'],
      dtype='object')

In [35]:
airbnb_new=airbnb[['id','city', 'neighbourhood', 'latitude', 'longitude', 'first_review']]

,id,city,neighbourhood,latitude,longitude
0,6901257,NYC,Brooklyn Heights,40.696524,-73.991617
1,6304928,NYC,Hell's Kitchen,40.766115,-73.989040
2,7919400,NYC,Harlem,40.808110,-73.943756
3,13418779,SF,Lower Haight,37.772004,-122.431619
4,3808709,DC,Columbia Heights,38.925627,-77.034596
...,...,...,...,...,...
74106,14549287,NYC,Williamsburg,40.709025,-73.939405
74107,13281809,LA,Hermosa Beach,33.871549,-118.396053
74108,18688039,NYC,Williamsburg,40.706749,-73.942377
74109,17045948,NYC,West Village,40.738535,-74.000157


In [38]:
airbnb_new.to_csv("Train reduced.csv", index=False)